### Important points to note :
Hashlib contains SHA-256 which will be used to calculate **block_hash**

In [104]:
import hashlib
import os, sys
from typing import List, Optional

### Transaction

#### Attributes

**from_account** : Account from which money was sent.

**to_account** : Account to which money is credited.

**amount** : Amount transferred.

**incentive** : transactions often have some incentive to be added to the next block like some transaction fees.

In [105]:
class Transaction:
    # Constructor
    def __init__(self,from_account: str,to_account: str,amount: int,incentive: int):
        self.from_account = from_account
        self.to_account = to_account
        self.amount = amount
        self.incentive = incentive
    

#### Function to Calculate Hash

In [106]:
def calculate_hash(data: bytes)->str:
    ans =  hashlib.sha3_256(data).hexdigest()
    #print("Calculated hash value = " + ans)
    return ans

### Merkle Tree

#### Attributes

**hash_value** : if both children, hash(node) = hash(hash(l)+hash(r)). else, hash(node) = hash(left)

**left** : Pointer to left

**right** : Pointer to right

In [107]:
class MerkleNode:
    # For leaf nodes, left and right is None
    def __init__(self,hash_value: str,left: Optional['MerkleNode'] = None,right: Optional['MerkleNode'] = None):
        self.hash_value = hash_value
        self.left = left
        self.right = right
        
    # Creating leaf node
    @staticmethod
    def create_leaf_node(t: Transaction)->'MerkleNode':
        hash_input = (t.from_account + str(t.incentive) + t.to_account + str(t.amount)).encode('utf-8')
        #print("hash of " + t.from_account + "-" + str(t.incentive) + "-" + t.to_account + "-" + str(t.amount))
        return MerkleNode(calculate_hash(hash_input))
        
    # Creating non-leaf nodes
    @staticmethod
    def create_internal_node(l: 'MerkleNode', r: Optional['MerkleNode'])->'MerkleNode':
        if r:
            #print("Hash value of internal node with left " + l.hash_value + " and right " + r.hash_value)
            hash_input = (l.hash_value + r.hash_value).encode('utf-8')
            return MerkleNode(calculate_hash(hash_input),l,r)  
        else:
            #print("Hash value of internal node with left " + l.hash_value)
            return MerkleNode(l.hash_value)
        
          

### Build Function

In [108]:
def build_merkle_tree(transactions: List[Transaction])->Optional['MerkleNode']:
    # no transactions present in list
    if not transactions:
        return None
    '''
    create a list of leaf nodes using the transactions
    [ t1 t2 t3 t4 ... tn ]
    then create the tree
    '''
    nodes = [MerkleNode.create_leaf_node(t) for t in transactions]

    while(len(nodes)>1):
        new_nodes = []
        for i in range(0,len(nodes),2):
            # we can add left and right
            if i < len(nodes) - 1 :
                new_nodes.append(MerkleNode.create_internal_node(nodes[i],nodes[i+1]))
            # can only add one of them
            else:
                new_nodes.append(MerkleNode.create_internal_node(nodes[i],None))
        nodes = new_nodes

    return nodes[0]
            
                

    
    
    
        
    


    

### Function to create the entire merkle tree and return the root's hash value

In [109]:
def get_merkle_root(t: List[Transaction])->str:
    root = build_merkle_tree(t)
    return root.hash_value if root else "0"

## Blocks

#### Attributes

**block_number**: current block number

**merkle_root**: root of merkle_tree

**block_hash**: Hash(prev_block_hash+curr_block_number+current_merkle_root).(utf-8)

**prevBlockHash**: Hash(prev_block_hash+curr_block_number+current_merkle_root)

**transactions**: array of transactions (at most 3)

**nonce**: chain follows a custom Proof Of Work where a number called nonce is added to block properties such that Computed_hash(CH) has zero in the last hexadecimal digit.
**CH** = #(block_hash+to_string(nonce)).

**miner_id**: id of miner that seals the block

In [110]:
class Block:
    #Constructor
    def __init__(self,block_number: int,prev_block_hash: str,transaction_list: List[Transaction],miner_id: str):
        self.block_number = block_number
        self.prev_block_hash = prev_block_hash
        self.transaction_list = transaction_list
        self.merkle_root = get_merkle_root(transaction_list)
        self.block_hash = calculate_hash((prev_block_hash+str(block_number)+self.merkle_root).encode('utf-8'))
        nonce = 0
        while True:
            computed_hash = calculate_hash((self.block_hash+str(nonce)).encode('utf-8'))
            if computed_hash[-1] == "0":
                self.nonce = nonce
                break
            nonce += 1
        self.miner_id = miner_id

### Miner

#### Attributes

**id**: identifier for the miner

**computation_score**: computation score for the miner

**block_hash_score_array**: data structure used to evaluate the performance or effectiveness of different hash attempts made by the miner when solving a Proof-of-Work (PoW) puzzle in blockchain mining.

In [111]:
class Miner:
    #Constructor
    def __init__(self,id: str,computation_score: int, block_hash_score_array: List[int]):
        self.id = id
        self.computation_score = computation_score
        self.block_hash_score_array = block_hash_score_array

#### Prints Miner Details

In [112]:
def get_miner_details(miners: List[Miner]):
    for m in miners:
        print(m.id)
        print(m.computation_score)
        print(m.block_hash_score_array)

### Execute Transactions

In [113]:
def execute_transaction(t: Transaction, balances: dict)->bool:
    balance_of_from = balances.get(t.from_account,0)
    balance_of_to = balances.get(t.to_account,0)
    # valid transaction only if balance of x exceeds amount being spent
    if balance_of_from >= t.amount:
        balances[t.from_account] = balances.get(t.from_account,0) - t.amount
        balances[t.to_account] = balances.get(t.to_account,0) + t.amount
        return True
    return False

### Prints Transaction Details

In [114]:
def get_transaction_details(transactions: List[Transaction]):
    all_ts = []
    for t in transactions:
        vals = [t.from_account,t.to_account,t.amount,t.incentive]
        all_ts.append(vals)
    print(all_ts)

### Finds the miner with maximum BSS

In [115]:
def get_miner_with_max_bss(miner_list: List[Miner], block_number: int):
    # find the miner
    block_sealing_scores = [ m.computation_score*m.block_hash_score_array[block_number%8] for m in miner_list]
    max_index = block_sealing_scores.index(max(block_sealing_scores))
    miner_id = miner_list[max_index].id

    return miner_id
    

# MAIN FUNCTION
    

In [122]:
def main():

    blocks = []
    block_number = 1
    prev_block_hash = "0"
    current_transactions = []
    max_transactions_in_block = 4
    
    # get the account details of every individual
    n = int(input())
    balances = {}
    for i in range(n):
        account,balance = str(input()).split()
        balances[account] = int(balance)

    # get every transaction details
    number_of_transactions = int(input())
    transactions = []
    for i in range(number_of_transactions):
        from_account,to_account,amount,incentive = str(input()).split()
        transactions.append(Transaction(from_account,to_account,int(amount),int(incentive)))

    # get block reward
    block_reward = int(input())
    
    # get miner details
    number_of_miners = int(input())
    miner_list = []
    for i in range(number_of_miners):
        combined = str(input()).split()
        identifier = str(combined[0])
        computation_score = int(combined[1])
        block_hash_score_array = [int(x) for x in combined[2:]]
        miner_list.append(Miner(identifier,computation_score,block_hash_score_array))
    
    # sort based on decreasing order of incentive and increasing order of receiver (to_account)
    transactions = sorted(transactions,key = lambda t:(-t.incentive,t.to_account))
    
    # now create the blockchain
    for t in transactions:
        # if valid transaction
        if execute_transaction(t,balances):
            current_transactions.append(t)
            if len(current_transactions) == max_transactions_in_block:

                # call get_miner_id function
                miner_id = get_miner_with_max_bss(miner_list,block_number)
                
                # create block and add to list of blocks
                block = Block(block_number,prev_block_hash,current_transactions,miner_id)
                prev_block_hash = block.block_hash
                current_transactions = []
                block_number = block_number + 1
                blocks.append(block)

                # update balance
                balances[miner_id] += block_reward


 
    if len(current_transactions) != 0:
        
        # call get_miner_id function
        miner_id = get_miner_with_max_bss(miner_list,block_number)
        
        block = Block(block_number,prev_block_hash,current_transactions,miner_id)
        blocks.append(block)

        # update balance
        balances[miner_id] += block_reward

    for b in blocks:
        print(b.block_number)
        print(b.block_hash)
        get_transaction_details(b.transaction_list)
        print(b.merkle_root)
        print(str(b.nonce) + " " + b.miner_id)

  
        
            



if __name__ == "__main__":
    main()

        

 7
 B 35
 L 20
 O 12
 C 40
 K 13
 E 100
 D 50
 11
 E B 10 12
 O K 20 9
 D B 3 12
 L O 25 3
 B K 100 8
 K L 15 7
 L C 12 1
 L D 25 12
 C E 3 8
 B C 8 9
 C O 5 10
 10
 3
 L 80 27 22 23 25 28 20 29 24
 E 30 77 16 20 26 19 60 22 15
 O 50 27 38 23 25 28 20 48 24


1
e85b466dc8244a2f46a8fa26adc852359e9abedfee88a5222e176d254c4f500a
[['E', 'B', 10, 12], ['D', 'B', 3, 12], ['C', 'O', 5, 10], ['B', 'C', 8, 9]]
e3785c5ca960fc032da650adb4e52d5ffbdfdcd63bca50b83e82e1d7e5a3ce03
0 O
2
746ffaccd9a4ce2d4397cb7d90ef648cbbad23a57d6fb12db0783df265927d6e
[['O', 'K', 20, 9], ['C', 'E', 3, 8], ['K', 'L', 15, 7], ['L', 'O', 25, 3]]
06b810c7e67eb1fb1106661f47b2b2d27226459826d9b2a4853cc399c01aba48
4 L
3
b65fa9c290b863dd1b11ab8a420b27e5a4a916cd76d3696158df37c337af27e4
[['L', 'C', 12, 1]]
e216d3b5e508e470da1f572d49cf9628c34a1dd7310653c40ebdb15b5ec847d2
10 L
